In [ ]:
import mysql.connector
import sys

# ----------------------------
# Connect to Database
# ----------------------------


def get_connection():
    try:
        conn = mysql.connector.connect(
            host="localhost",
            user="root",
            password="//PASSWORD//",
            database="FINAL_URP"
        )
        print("Connected to MySQL database successfully!")
        return conn
    except mysql.connector.Error as err:
        print(f"Error connecting to MySQL: {err}")
        sys.exit(1)


# ----------------------------
# Run Test Helper
# ----------------------------
def run_test(name, action_query=None, action_params=None, fetch_query=None, fetch_params=None, success_message=None):
    conn = get_connection()
    cursor = conn.cursor(dictionary=True)
    print(f"\n=== Test: {name} ===")
    try:
        # Action query (INSERT/UPDATE/DELETE)
        if action_query:
            if action_params:
                cursor.execute(action_query, action_params)
            else:
                cursor.execute(action_query)
            conn.commit()
            if success_message:
                print(success_message)

        # Fetch query (SELECT)
        if fetch_query:
            if fetch_params:
                cursor.execute(fetch_query, fetch_params)
            else:
                cursor.execute(fetch_query)
            result = cursor.fetchall()
            if result:
                for row in result:
                    print(row)
            else:
                print("No results found.")

    except mysql.connector.Error as e:
        print("FAIL:", e)
        conn.rollback()
    finally:
        cursor.close()
        conn.close()


# ----------------------------
# CRUD TESTS
# ----------------------------
def test_crud():
    # INSERT Test 1
    run_test(
        name="INSERT Member Test 1",
        action_query="""
            INSERT INTO Member(name, email, role, dept_id)
            VALUES (%s, %s, %s, %s)
        """,
        action_params=('User1', 'user1@sjsu.edu', 'Student', 1),
        fetch_query="SELECT email, role FROM Member WHERE email=%s",
        fetch_params=('user1@sjsu.edu',),
        success_message="Data inserted successfully!"
    )

    # INSERT Test 2
    run_test(
        name="INSERT Member Test 2",
        action_query="""
            INSERT INTO Member(name, email, role, dept_id)
            VALUES (%s, %s, %s, %s)
        """,
        action_params=('User2', 'user2@sjsu.edu', 'Student', 2),
        fetch_query="SELECT email, role FROM Member WHERE email=%s",
        fetch_params=('user2@sjsu.edu',),
        success_message="Data inserted successfully!"
    )

    # READ Test
    run_test(
        name="READ Member Test 1",
        fetch_query="SELECT email, role FROM Member WHERE email=%s",
        fetch_params=('user1@sjsu.edu',)
    )

    # UPDATE Test 1
    run_test(
        name="UPDATE Member 1",
        action_query="UPDATE Member SET role=%s WHERE email=%s",
        action_params=('Student', 'user1@sjsu.edu'),
        fetch_query="SELECT role FROM Member WHERE email=%s",
        fetch_params=('user1@sjsu.edu',),
        success_message="Data updated successfully!"
    )

    # UPDATE Test 2 (fixed fetch_query to check correct column)
    run_test(
        name="UPDATE Member 2",
        action_query="UPDATE Member SET student_id=%s WHERE email=%s",
        action_params=(3, 'user2@sjsu.edu'),
        fetch_query="SELECT student_id FROM Member WHERE email=%s",
        fetch_params=('user2@sjsu.edu',),
        success_message="Data updated successfully!"
    )

    # DELETE Tests
    run_test(
        name="DELETE Member Test 1",
        action_query="DELETE FROM Member WHERE email=%s",
        action_params=('user1@sjsu.edu',),
        fetch_query="SELECT COUNT(*) as count FROM Member WHERE email=%s",
        fetch_params=('user1@sjsu.edu',),
        success_message="Data deleted successfully!"
    )

    run_test(
        name="DELETE Member Test 2",
        action_query="DELETE FROM Member WHERE email=%s",
        action_params=('user2@sjsu.edu',),
        fetch_query="SELECT COUNT(*) as count FROM Member WHERE email=%s",
        fetch_params=('user2@sjsu.edu',),
        success_message="Data deleted successfully!"
    )


# ----------------------------
# COMPLEX QUERY TESTS
# ----------------------------
def test_complex_queries():
    # JOIN Test 1
    run_test(
        name="JOIN Test 1 — Members with Departments",
        fetch_query="""
            SELECT m.name, d.dept_name
            FROM Member m
            JOIN Department d ON m.dept_id=d.dept_id
            LIMIT 5;
        """
    )

    # JOIN Test 2
    run_test(
        name="JOIN Test 2 — Members with Departments, Dept ID 1",
        fetch_query="""
            SELECT m.name, d.dept_name
            FROM Member m
            JOIN Department d ON m.dept_id=d.dept_id
            WHERE d.dept_id=1
            LIMIT 5;
        """
    )

    # SUBQUERY Test 1
    run_test(
        name="SUBQUERY Test 1 — Members in Largest Department",
        fetch_query="""
            SELECT * FROM Member
            WHERE dept_id = (
                SELECT dept_id
                FROM Member
                GROUP BY dept_id
                ORDER BY COUNT(*) DESC
                LIMIT 1
            )
            LIMIT 5;
        """
    )

    # SUBQUERY Test 2
    run_test(
        name="SUBQUERY Test 2 — Members in Smallest Department",
        fetch_query="""
            SELECT * FROM Member
            WHERE dept_id = (
                SELECT dept_id
                FROM Member
                GROUP BY dept_id
                ORDER BY COUNT(*) ASC
                LIMIT 1
            )
            LIMIT 5;
        """
    )

    # AGGREGATE / GROUP BY Test 1
    run_test(
        name="AGGREGATE Test 1 — Count Members per Department",
        fetch_query="""
            SELECT dept_id, COUNT(*) as total_members FROM Member
            GROUP BY dept_id
            LIMIT 5;
        """
    )

    # AGGREGATE / GROUP BY Test 2
    run_test(
        name="AGGREGATE Test 2 — Count Faculty Members per Department",
        fetch_query="""
            SELECT dept_id, COUNT(*) as faculty_count FROM Member
            WHERE role='Faculty'
            GROUP BY dept_id
            LIMIT 5;
        """
    )


# ----------------------------
# MAIN
# ----------------------------
if __name__ == "__main__":
    print("\n========== RUNNING CRUD TESTS ==========")
    test_crud()  # Run all CRUD tests

    print("\n========== RUNNING COMPLEX QUERY TESTS ==========")
    test_complex_queries()  # Run all complex queries
